![卷积输出](imgs/6_3填充和步幅1.png)
##### 不加padding和stride，输出维度为：N - K + 1
还有什么因素会影响输出的大小呢？

节我们将介绍填充（padding）和步幅（stride）。假设以下情景： 有时，在应用了连续的卷积之后，我们最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于 1 所导致的。比如，一个 240×240 像素的图像，经过 10 层 5×5 的卷积后，将减少到 200×200 像素。如此一来，原始图像的边界丢失了许多有用信息。而填充是解决此问题最有效的方法。 有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。步幅则可以在这类情况下提供帮助。

### 6.3.1. 填充
在应用多层卷积时，我们常常会丢失边缘像素。
由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素、 但随着我们应用许多连续卷积层，累积丢失的像素数就多了。 解决这个问题的简单方法即为填充（padding）：在输入图像的边界填充元素（通常填充元素是 0 ）。
![填充后的尺寸](imgs/6_3填充和步幅1.png)
##### padding后的尺寸：N - K + 1 + P
##### 因此，我们只需要设置padding的值为 K - 1，就可以使得，输入和输出具有相同的维度。
这样，我们在构建网络的时候更容易的预测每个图形的输出形状。

##### 假设卷积核的大小为奇数K，我们在其周围填充P/2行。
卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。
选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。

此外，使用奇数核和填充也提供了书写上的便利。
对于任何二维张量X，当满足：
1. 内核的大小是奇数
2. 所有边的填充行数与列数相同
3. 输出与输入具有相同的高度与宽度

输出Y[i, j]是通过以输入X[i, j]为中心，与卷积核进行互相关计算得到的。

比如，在下面的例子中，我们创建一个高度和宽度为3的二维卷积层，并在所有侧边填充1个像素。给定高度和宽度为8的输入，则输出的高度和宽度也是8。

In [1]:
import torch
from torch import nn
# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    r = Y.reshape(Y.shape[2:])
    return r

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [2]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 6.3.2. 步幅
在计算互相关时，卷积窗口从输入张量的左上角开始，向下和向右滑动。
在前面的例子中，我们默认每次滑动一个元素。
但是，有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素。

我们将每次滑动元素的数量称为步幅（stride）。

![步幅](imgs/6_3_2步幅1.png)

##### 通常，输出的形状为⌊ (N - K + P + S ) / S ⌋
如果我们把P = K - 1，则
##### ⌊ (N - K + K - 1 + S ) / S ⌋ = ⌊ ( N + S - 1 ) / S⌋
更进一步，如果W和H可以被S整除，则输出形状为
$$( N / S)$$

下面，我们将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半。

In [3]:
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

In [5]:
conv = nn.Conv2d(3, 4, kernel_size=3, padding=2, stride=8)
X = torch.rand(1, 3, 64, 64)# B C W H
X.shape, conv(X).shape

(torch.Size([1, 3, 64, 64]), torch.Size([1, 4, 9, 9]))

如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为 (n/s) 。
可以看出，P=K-1，输出形状为 N / S + 1